In [ ]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k
from src.data.features import feature_engineering
from src.data.utils import build_rank_input
from src.models.tuner import BayesianSearch

In [ ]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [ ]:
# load and prepare data
dataframes = load_data(config=config['data_loader'])
dataframes = prepare_data(dataframes=dataframes)

In [ ]:
# train-test split
df_train, df_test = leave_last_k(df=dataframes['data'], config=config['model_selection'])
df_train, df_valid = leave_last_k(df=df_train, config=config['model_selection'])

In [ ]:
user_item_features = feature_engineering(
    dataframes={'user': dataframes['user'], 'item': dataframes['item'], 'data': df_train}
    )

df_train, df_valid = [
    build_rank_input(ratings=df, features=user_item_features) for df in (df_train, df_valid)
    ]

In [ ]:
# perform bayesian search
searcher = BayesianSearch(config['model_selection']['hyper_params'], algorithm='XGBRanker')

def objective(trial) -> float:
    return searcher.fit(df_train, df_valid, trial)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

In [ ]:
print("Best trial:")
print(study.best_trial.params)